In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri May 26 13:08:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.8 MB/s eta 0:00:00


In [3]:
import os, torch
import pandas as pd
import time
from torch.utils.data import TensorDataset, SequentialSampler, DataLoader, Dataset
from transformers import BertTokenizer, BertModel
import numpy as np
from torch import nn
from tqdm import tqdm
from torch.optim import SGD
from torch.nn.functional import softmax
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset
from transformers import AdamW

In [4]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [5]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/licenta/')

Mounted at /content/drive


In [6]:
print(os.listdir('./'))

['get_data_after_bert.py', 'run_prin_bert_freezed.ipynb', 'data', 'cls_output', 'logs', 'first_try.ipynb', 'modele']


In [7]:
# def loss_fn(outputs, targets):
#     return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [8]:
def get_all_data_from_filename(full_filename):
    df = pd.read_csv(full_filename, delimiter='\t', header=None, names=['iid','src','native_speaker','original','dest','text','direct','label'])
    return df

In [9]:
def get_text_and_label(df):
    return df.text.values[1:], df.label.values[1:]

In [10]:
def import_model(name='bert-base-uncased'):
    token = BertTokenizer.from_pretrained(name)
    mod = BertModel.from_pretrained(name)
    return token, mod

In [11]:
# class DatasetBERT(Dataset):
#     def __init__(self, X, Y, tokenizer, max_len, device):
#         Y = np.stack(Y).astype(float)
#         self.X = X
#         self.Y = torch.tensor(Y, device=device).float()
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return self.X.shape[0]
    
#     def __getitem__(self, index):
#       text = self.X[index]
#       inputs = self.tokenizer.encode_plus(
#                             text,                      # Sentence to encode.
#                             add_special_tokens = True, # Add '[CLS]' and '[SEP]'
#                             max_length = self.max_len,           # Pad & truncate all sentences.
#                             pad_to_max_length = True,
#                             return_attention_mask = True,   # Construct attn. masks.
#                             return_tensors = 'pt',     # Return pytorch tensors.
#                        )
#       ids = inputs['input_ids']
#       # print(f"class - input ids shape = {ids.shape}")
#       mask = inputs['attention_mask']
#       # print(f"class - input mask shape = {mask.shape}")
#       token_type_ids = inputs["token_type_ids"]
#       # print(f"class - input token type shape = {token_type_ids.shape}")
#       return {
#             'ids': torch.tensor(ids, dtype=torch.long),
#             'mask': torch.tensor(mask, dtype=torch.long),
#             'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
#             'labels': torch.tensor(self.Y[index], dtype=torch.float)
#         }

In [12]:
class MLP(nn.Module):
  def __init__(self, mod):
    super().__init__()
    self.bert = mod
    self.layer1 = nn.Linear(768, 512)
    self.layer2 = nn.Linear(512, 128)
    self.layer3 = nn.Linear(128, 2)
    self.activation = nn.Tanh()
    # self.drop = nn.Dropout1d(p=0.1)

  def forward(self, ids, mask, token_type_ids):
    out= self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)['pooler_output']
    out = self.activation(out)
    out = self.activation(self.layer1(out))
    # out = self.drop(out)
    out = self.layer2(out)
    out = self.layer3(out)
    return out

In [13]:
TOKENIZER, model_bert = import_model()
MAX_LEN = 256
BATCH_SIZE = 32

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
# torch.save(model_bert, './danibaciutest')

In [15]:
NUM_EPOCHS = 5

In [16]:
def transform_input(tokenizer, X, Y, max_len):
  print(X.shape)
  print(Y.shape)
  print(type(Y[0]))

  input_ids = []
  attention_masks = [] 
  token_type_ids = []

  for sent in X:

    encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_len,           # Pad & truncate all sentences.
                            padding = 'max_length',
                            truncation = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_token_type_ids=True,
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
    
    input_ids.append(encoded_dict["input_ids"])
    attention_masks.append(encoded_dict["attention_mask"])
    token_type_ids.append(encoded_dict["token_type_ids"])

  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  token_type_ids = torch.cat(token_type_ids, dim=0)
  labels = torch.tensor([int(a) for a in Y])

  data = TensorDataset(input_ids, attention_masks, token_type_ids, labels)
  return data


In [17]:
def get_dataloader_from_filename_batch(filename, batch_size, tokenizer, max_len, shuffle = True):
  date_train_X, date_train_Y = get_text_and_label(get_all_data_from_filename(filename))

  dataset_train = transform_input(tokenizer, date_train_X, date_train_Y, max_len)

  dataloader_train = DataLoader(dataset_train, batch_size, shuffle = shuffle)
  return dataloader_train

In [18]:
# dataloader_train = get_dataloader_from_filename_batch('./data/mono_en_de_train.tsv', BATCH_SIZE, TOKENIZER, MAX_LEN)
dataloader_test = get_dataloader_from_filename_batch('./data/mono_en_de_test.tsv', BATCH_SIZE, TOKENIZER, MAX_LEN, False)

(6344,)
(6344,)
<class 'str'>


In [19]:
def compute_accuracy(predictions, labels):
    """
    Compute accuracy given the predictions of a binary classifier and the 
    ground truth label.
    predictions: list of model predictions (0 or 1)
    labels: list of ground truth labels (0 or 1)
    """
    num_correct = len([(p,l) for (p,l) in zip(predictions,labels) if p==l])
    epoch_accuracy = num_correct/len(labels)
    
    return epoch_accuracy

In [20]:
# def train_epoch(model, train_dataloader, loss_crt, optimizer, device) :

#     model.train()
#     epoch_loss = 0.0
#     num_batches = len(train_dataloader)
#     predictions = []
#     labels = []

#     for idx, batch in enumerate(train_dataloader):
#         batch_ids, batch_attention, batch_token_type_id, batch_labels = batch

#         # move data to GPU
#         batch_ids = batch_ids.to(device)
#         batch_attention = batch_attention.to(device)
#         batch_token_type_id = batch_token_type_id.to(device)
#         batch_labels = batch_labels.to(device)


#         output = model(batch_ids, batch_attention,batch_token_type_id)

#         batch_predictions = torch.argmax(output, dim=1)
        
#         predictions += batch_predictions.tolist()
#         labels += batch_labels.squeeze().tolist()

#         loss = loss_crt(output, batch_labels.squeeze())
#         loss_scalar = loss.item()

#         loss.backward()

#         optimizer.step()
#         model.zero_grad()

#         epoch_loss += loss_scalar

#     epoch_loss = epoch_loss/num_batches

#     return epoch_loss, predictions, labels


In [21]:
def eval_epoch(model, val_dataloader, loss_crt, device):
    model.eval()
    epoch_loss = 0.0
    num_batches = len(val_dataloader)
    predictions = []
    labels = []
    with torch.no_grad():
        for idx, batch in enumerate(val_dataloader):
            batch_ids, batch_attention, batch_token_type_id,batch_labels = batch

            # move data to GPU
            batch_ids = batch_ids.to(device)
            batch_attention = batch_attention.to(device)
            batch_token_type_id = batch_token_type_id.to(device)
            batch_labels = batch_labels.to(device)
            
            output = model(batch_ids, batch_attention,batch_token_type_id)

            # batch_size
            batch_predictions = torch.argmax(output, dim=1)
            predictions += batch_predictions.tolist()
            labels += batch_labels.squeeze().tolist()

            loss = loss_crt(output, batch_labels.squeeze())
            loss_scalar = loss.item()

            epoch_loss += loss_scalar

    epoch_loss = epoch_loss/num_batches

    return epoch_loss, predictions, labels


In [22]:
# os.listdir('./modele/etapa1/')

In [23]:
def run(test_dataloader, _learning_rate, name):

    loss_fn = torch.nn.CrossEntropyLoss()

    val_losses = []
    val_accuracies = []

    for i in range(NUM_EPOCHS):

        complet_name = './modele/ger/etapa1/' + name + '_' + str(_learning_rate) + '_' + str(i)
        # print(f"S-a incarcat modelul cu numele {complet_name}")
        model = MLP(model_bert)
        model.to(device)
        model = torch.load(complet_name)
        # model.to(device)
        # print(f"S-a incarcat modelul cu numele {complet_name}")

        optimizer = AdamW(model.parameters(), lr=_learning_rate)

        dev_epoch_loss, dev_predictions, dev_labels = eval_epoch(model, test_dataloader, loss_fn, device)

        dev_acc = compute_accuracy(dev_predictions, dev_labels)

        val_losses.append(dev_epoch_loss)
        val_accuracies.append(dev_acc)

        print("Test loss: ", dev_epoch_loss)
        print("Acc test: ", dev_acc)

    return val_losses


In [24]:
import time
k = 6 # CHANGE THIS !!!!!!
for _learning_rate in [4*0.00001, 2*0.00001] :
  print('\n\n-----------------------------------\n\n')
  print(f'learning rate = {_learning_rate}')
  started_at = time.time()
  val_losses = run(dataloader_test, _learning_rate, 'model' + str(k))
  completed_at = time.time()

  print(f' For LR={_learning_rate} time elapsed is {int(completed_at - started_at)}')




-----------------------------------


learning rate = 4e-05


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Test loss:  0.2829320089114671
Acc test:  0.8816204287515763
Test loss:  0.23925730184974833
Acc test:  0.9046343001261034
Test loss:  0.3366202077730067
Acc test:  0.9036885245901639
Test loss:  0.37161404757535826
Acc test:  0.9014817150063051
Test loss:  0.4131156715415108
Acc test:  0.9033732660781841
 For LR=4e-05 time elapsed is 168


-----------------------------------


learning rate = 2e-05
Test loss:  0.41277075349060754
Acc test:  0.9066834804539723
Test loss:  0.5032654620335778
Acc test:  0.9040037831021438
Test loss:  0.4928675162784074
Acc test:  0.9036885245901639
Test loss:  0.3677506497410306
Acc test:  0.9068411097099621
Test loss:  0.5564702905611735
Acc test:  0.9027427490542245
 For LR=2e-05 time elapsed is 154
